In [1]:
from pathlib import Path
import sys
import pickle

import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt

%matplotlib inline
plt.style.use("default")

In [2]:
sys.path.append(
    "/data/yonatan/myproject/scripts/indicator_config/indicator_configuration/modules",
)
from convex_optimization_mod import ConvexOptMod as ConvexOpt
from utils import tol, eps

In [3]:
case = 14  # 14-bus case

WORK_DIR = Path().absolute()
DATA_DIR = WORK_DIR / "data" / f"IEEE{case}"
FIM_DIR = DATA_DIR / "FIMs"
RESULT_DIR = WORK_DIR / "results" / f"IEEE{case}"
if not RESULT_DIR.exists():
    RESULT_DIR.mkdir(parents=True)

In [4]:
# Model information
nparams = 2 * case

# Remove optimal buses from the original problem

## Setup

In [5]:
# Configurations
configs_all = list(np.arange(case) + 1)
configs_remove = [2, 6, 9]
configs = list(set(configs_all) - set(configs_remove))
nconfigs = len(configs)
config_ids = np.array([f"bus{ii}" for ii in configs])
config_ids

array(['bus1', 'bus3', 'bus4', 'bus5', 'bus7', 'bus8', 'bus10', 'bus11',
       'bus12', 'bus13', 'bus14'], dtype='<U5')

In [6]:
# Load configuration FIMs
fim_configs_tensor = np.empty((nconfigs, nparams, nparams))
for ii, bus in enumerate(configs):
    J = np.load(FIM_DIR / f"jacobian_bus{bus}.npy")
    I = J.T @ J
    fim_configs_tensor[ii] = I

## Convex optimization

In [7]:
# Convex optimization
# Settings
cvx_tol = eps ** 0.75
lambda_tol = 1e-5
tol_mat = np.diag(np.ones(nparams)) * lambda_tol
print("Tolerance:", cvx_tol)
print("Lower bound:", lambda_tol)

solver = dict(verbose=True, solver=cp.SDPA, epsilonStar=cvx_tol)

# plt.figure()
wopt = np.ones(nconfigs)
colors = ["r", "g", "b", "k"]
for ii in range(4):
    cvxopt = ConvexOpt(fim_configs_tensor, np.array(config_ids), tol_mat, wopt + 1e-15)

    try:
        # Solve
        cvxopt.solve(solver=solver)
    except Exception:
        break

    wopt = cvxopt.result["wm"]
    dual = cvxopt.result["dual_wm"]
    print("Violation:", cvxopt.constraints[1].violation())
    # print("Weights:", wm)

#     plt.plot(wopt, c=colors[ii], label=ii)
#     plt.plot(dual, c=colors[ii], ls="--")

# plt.yscale("log")
# plt.xticks(range(nconfigs), config_ids, rotation=90)
# plt.ylabel("Weights")
# plt.legend(title="Iteration", bbox_to_anchor=(1, 1))
# plt.show()

Tolerance: 1.8189894035458565e-12
Lower bound: 1e-05
                                     CVXPY                                     
                                     v1.3.2                                    
(CVXPY) May 19 02:35:56 PM: Your problem has 11 variables, 2 constraints, and 0 parameters.
(CVXPY) May 19 02:35:56 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) May 19 02:35:56 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) May 19 02:35:56 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) May 19 02:35:56 PM: Compiling problem (target solver=SDPA).
(CVXPY) May 19 02:35:56 PM: Reduction chain

/home/byu.local/yonatank/.local/lib/python3.7/site-packages/cvxpy/problems/problem.py:1388: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "



phase.value  = pdFEAS    
   Iteration = 17
          mu = +1.2978914628387435e-08
relative gap = +7.9171355187374255e-07
        gap  = +7.9171355187374255e-07
     digits  = +2.2471747244991040e+00
objValPrimal = +1.4027172761758006e-04
objValDual   = +1.3948001406570632e-04
p.feas.error = +8.9998841712457534e-13
d.feas.error = +9.8910421953563032e-12
total time   = 0.120386
  main loop time = 0.120382
      total time = 0.120386
file  check time = 0.000000
file change time = 0.000004
file   read time = 0.000000
Converting optimal solution to CLP format
SDPA end at [Sun May 19 14:35:56 2024]
SDPA start at [Sun May 19 14:35:56 2024]
NumThreads  is set as 64
Schur computation : DENSE 
Converted to SDPA internal data / Starting SDPA main loop
   mu      thetaP  thetaD  objP      objD      alphaP  alphaD  beta 
Start: getCLPresult 0 1.0e+04 1.0e+00 1.0e+00 -0.00e+00 +2.80e-02 9.0e-01 8.3e-01 2.00e-01
 1 1.7e+03 1.0e-01 1.7e-01 +1.38e+03 +3.73e-02 9.0e-01 7.1e-01 2.00e-01
 2 4.5e+02 1.0e

(CVXPY) May 19 02:35:57 PM: Optimal value: 3.741e+00
(CVXPY) May 19 02:35:57 PM: Compilation took 7.172e-02 seconds
(CVXPY) May 19 02:35:57 PM: Solver (including time spent in interface) took 3.399e-01 seconds
Violation: 4.431743675028497e-08
                                     CVXPY                                     
                                     v1.3.2                                    
(CVXPY) May 19 02:35:57 PM: Your problem has 11 variables, 2 constraints, and 0 parameters.
(CVXPY) May 19 02:35:57 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) May 19 02:35:57 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) May 19 02:35:57 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-

### Extracting the weights

In [8]:
# # Plot the weights
# plt.figure()
# plt.title(f"Eigenvalue lower bound: {lambda_tol:0.3e}")
# plt.plot(wopt, label="weights")
# plt.plot(dual, label="dual weights")
# plt.yscale("log")
# plt.xticks(range(nconfigs), config_ids, rotation=90)
# plt.legend()
# plt.show()

In [9]:
# Get the optimal buses
idx_wopt = np.where(wopt > 1e-12)[0]  # Index to the optimal buses
print("Optimal buses:")
_ = [print(f'{config_ids[ii]} \t {wopt[ii]}') for ii in idx_wopt]

Optimal buses:
bus3 	 1.1704499821279137e-07
bus5 	 0.0001359570325405998
bus7 	 7.133314592954387e-07
bus10 	 5.093727603667292e-05
bus13 	 6.152558307178295e-06


In [10]:
weighted_fims_configs = fim_configs_tensor * wopt.reshape((-1, 1, 1))
I = np.sum(weighted_fims_configs[idx_wopt], axis=0)
print("Eigenvalues:")
print(np.linalg.eigvalsh(I))

# plt.figure()
# cbound = np.max([-np.min(I), np.max(I)])
# plt.imshow(I, vmin=-cbound, vmax=cbound, cmap="bwr")
# plt.colorbar()
# plt.show()

Eigenvalues:
[9.96792536e-06 1.09752483e-05 1.30895101e-05 1.56022912e-05
 2.03738738e-05 2.69372551e-05 3.31680433e-05 3.67210258e-05
 5.53118656e-05 5.88633147e-05 8.86783936e-05 1.02235756e-04
 1.41601919e-03 1.58282367e-03 2.08101572e-03 2.32746810e-03
 3.05826228e-03 3.45923358e-03 3.89503882e-03 4.31008577e-03
 1.32876149e-02 1.51070480e-02 3.34136647e-02 3.75562093e-02
 1.15147251e-01 1.29582160e-01 4.18352252e-01 4.38506643e-01]


# Remove 1 step further

In [11]:
# Configurations
configs_remove = np.array(configs)[idx_wopt]
configs = list(set(configs) - set(configs_remove))
nconfigs = len(configs)
config_ids = np.array([f"bus{ii}" for ii in configs])
config_ids

array(['bus1', 'bus4', 'bus8', 'bus11', 'bus12', 'bus14'], dtype='<U5')

In [12]:
# Load configuration FIMs
fim_configs_tensor = np.empty((nconfigs, nparams, nparams))
for ii, bus in enumerate(configs):
    J = np.load(FIM_DIR / f"jacobian_bus{bus}.npy")
    I = J.T @ J
    fim_configs_tensor[ii] = I

In [13]:
# Convex optimization
# Settings
wopt = np.ones(nconfigs)
for ii in range(4):
    cvxopt = ConvexOpt(fim_configs_tensor, np.array(config_ids), tol_mat, wopt + 1e-15)

    try:
        # Solve
        cvxopt.solve(solver=solver)
    except Exception:
        break

    wopt = cvxopt.result["wm"]
    dual = cvxopt.result["dual_wm"]
    print("Violation:", cvxopt.constraints[1].violation())

                                     CVXPY                                     
                                     v1.3.2                                    
(CVXPY) May 19 02:35:57 PM: Your problem has 6 variables, 2 constraints, and 0 parameters.
(CVXPY) May 19 02:35:57 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) May 19 02:35:57 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) May 19 02:35:57 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) May 19 02:35:57 PM: Compiling problem (target solver=SDPA).
(CVXPY) May 19 02:35:57 PM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing -> 

Making result infomation...
.00e-01
14 1.7e-05 2.2e-17 1.1e-13 +3.92e+00 +3.92e+00 1.1e+00 9.9e-01 1.00e-01
15 1.8e-06 2.2e-17 5.1e-13 +3.92e+00 +3.92e+00 1.3e+00 1.0e+00 2.00e-01
16 3.1e-07 1.9e-17 1.1e-12 +3.92e+00 +3.92e+00 1.6e+00 1.0e+00 2.00e-01
17 5.5e-08 2.2e-17 5.9e-13 +3.92e+00 +3.92e+00 1.5e+00 1.0e+00 2.00e-01
18 9.6e-09 2.2e-17 1.6e-13 +3.92e+00 +3.92e+00 1.1e+00 1.0e+00 2.00e-01
19 1.9e-09 2.2e-17 8.2e-13 +3.92e+00 +3.92e+00 1.1e+00 1.0e+00 2.00e-01
20 3.4e-10 1.9e-17 1.1e-12 +3.92e+00 +3.92e+00 1.2e+00 1.0e+00 2.00e-01
Strange behavior : primal < dual :: line 158 in sdpa_solve.cpp
21 6.4e-11 2.2e-17 9.8e-14 +3.92e+00 +3.92e+00 9.0e-01 9.0e-01 1.00e-01
 SDPAP: Result
    SDPA.phase = pdFEAS
     iteration = 21
    convMethod = LMI
     frvMethod = split
  domainMethod = none
   rangeMethod = none
     primalObj = +3.9179703072192940e+00
       dualObj = +3.9179703042832328e+00
    dualityGap = +7.4938355196899999e-10
   primalError = +0.0000000000000000e+00
     dualError

(CVXPY) May 19 02:35:58 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) May 19 02:35:58 PM: Compiling problem (target solver=SDPA).
(CVXPY) May 19 02:35:58 PM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing -> SDPA
(CVXPY) May 19 02:35:58 PM: Applying reduction Dcp2Cone
(CVXPY) May 19 02:35:58 PM: Applying reduction CvxAttr2Constr
(CVXPY) May 19 02:35:58 PM: Applying reduction ConeMatrixStuffing
(CVXPY) May 19 02:35:58 PM: Applying reduction SDPA
(CVXPY) May 19 02:35:58 PM: Finished problem compilation (took 4.846e-02 seconds).
-------------------------------------------------------------------------------
                                Numerical solver              

In [14]:
# Get the optimal buses
idx_wopt = np.where(wopt > 1e-12)[0]  # Index to the optimal buses
print("Optimal buses:")
_ = [print(f'{config_ids[ii]} \t {wopt[ii]}') for ii in idx_wopt]

Optimal buses:
bus1 	 2.1157764378165252e-08
bus4 	 0.00016125191334759596
bus11 	 4.2448283246930655e-05
bus12 	 2.8722881156670066e-07
bus14 	 6.066826519146895e-06


In [15]:
weighted_fims_configs = fim_configs_tensor * wopt.reshape((-1, 1, 1))
I = np.sum(weighted_fims_configs[idx_wopt], axis=0)
print("Eigenvalues:")
print(np.linalg.eigvalsh(I))

Eigenvalues:
[9.94810727e-06 1.05422906e-05 1.21439424e-05 1.44620186e-05
 1.78800478e-05 2.07916491e-05 2.82284345e-05 3.06492058e-05
 4.22358167e-05 4.62257390e-05 1.26265960e-04 1.48649611e-04
 4.40700602e-04 5.05351733e-04 1.03289689e-03 1.16253647e-03
 2.18968739e-03 2.43997387e-03 4.91773027e-03 5.19688193e-03
 9.27795953e-03 1.04591213e-02 1.81712216e-02 1.93762049e-02
 1.19882250e-01 1.34826171e-01 5.42055086e-01 5.75489258e-01]
